# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch', 'AngleX', 'AngleY'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [1]:
%pylab inline
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics

ROBOT_POSE_DATA_DIR = 'robot_pose_data'

Populating the interactive namespace from numpy and matplotlib


In [2]:
classes = listdir(ROBOT_POSE_DATA_DIR)
print classes

['Left', 'Knee', 'Back', 'Right', 'Crouch', 'Sit', 'Frog', 'Stand', 'HeadBack', 'Belly', 'StandInit']


In [3]:
def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    # YOUR CODE HERE
    # i don't see anything that is missing
    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    data = pickle.load(open(filename))
    target = [i] * len(data)
    return data, target

In [4]:
# load all the data
all_data = []
all_target = []
# YOUR CODE HERE
for i in range(len(classes)):
    d, t = load_pose_data(i)
    all_data.extend(d)
    all_target.extend(t)

print 'total number of data', len(all_data)

total number of data 222


In [5]:
# shuffule data
permutation = np.random.permutation(len(all_data))
n_training_data = int(len(all_data) * 0.7)
training_data = permutation[:n_training_data]

## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [6]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [7]:
# YOUR CODE HERE
# training_data are just indices
t_data = []
t_target = []
for i in training_data:
    t_data.append(all_data[i])
    t_target.append(all_target[i])
clf.fit(t_data, t_target)

SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

### predicting

In [8]:
#clf.predict(all_data[-1]), all_target[-1] # Throws ERROR
clf.predict(all_data[-1:]), all_target[-1:] # from scikit-learn-intro

(array([10]), [10])

In [9]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

In [10]:
expected = []
predicted = []
# YOUR CODE HERE

# predict on training set
expected = t_target
predicted = clf.predict(t_data)

evaluate(expected, predicted)

Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        15
           1       1.00      1.00      1.00         7
           2       1.00      1.00      1.00        14
           3       1.00      1.00      1.00         7
           4       1.00      1.00      1.00        21
           5       1.00      1.00      1.00        21
           6       1.00      1.00      1.00         7
           7       1.00      1.00      1.00         9
           8       1.00      1.00      1.00         7
           9       1.00      1.00      1.00        12
          10       1.00      1.00      1.00        35

   micro avg       1.00      1.00      1.00       155
   macro avg       1.00      1.00      1.00       155
weighted avg       1.00      1.00      1.00       155


Confusion matrix:
[[15  0  0  0  0  0  0  0  0  0  0]
 [ 0  7  0  0  0  0  0  0  0  0  0]
 [ 0  0 14  0  0  0  0  0  0  0  0]
 [ 0  0  0  7  0  0  0  0  0  0  0]
 

## 4. Evaluate on the test data

In [11]:
expected = []
predicted = []
# YOUR CODE HERE

# predict on test set -> build test set
# shape of all_data is (222,10) (222 samples with 10 features each)
# len(training_data) is 155 -> 155 elements (with all their "columns") have to be deleted
# in order to use the remaining elements as test_data

# expected will get its elements from the subset of all_target
# predicted will get its elements from clf.predict

# list with every item has 10 as list
indices_to_delete = training_data

test_data = np.asarray(all_data)

test_data = np.delete(test_data, indices_to_delete, axis=0) # delete the 155 training_data_elements
# new shape: (67,10)

test_target = np.asarray(all_target)
test_target = np.delete(test_target, indices_to_delete, axis=0)

expected = test_target
predicted = clf.predict(test_data)

evaluate(expected, predicted)

Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         5
           1       1.00      1.00      1.00         3
           2       1.00      1.00      1.00         9
           3       1.00      0.75      0.86         4
           4       0.90      1.00      0.95         9
           5       1.00      1.00      1.00         5
           6       1.00      1.00      1.00         3
           7       1.00      1.00      1.00         2
           8       1.00      1.00      1.00         3
           9       1.00      1.00      1.00         7
          10       1.00      1.00      1.00        17

   micro avg       0.99      0.99      0.99        67
   macro avg       0.99      0.98      0.98        67
weighted avg       0.99      0.99      0.98        67


Confusion matrix:
[[ 5  0  0  0  0  0  0  0  0  0  0]
 [ 0  3  0  0  0  0  0  0  0  0  0]
 [ 0  0  9  0  0  0  0  0  0  0  0]
 [ 0  0  0  3  1  0  0  0  0  0  0]
 

## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [12]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'w'))

Then, in the application we can load the trained classifier again.

In [13]:
clf2 = pickle.load(open(ROBOT_POSE_CLF))
#clf2.predict(all_data[-1]), all_target[-1] # throws ERROR
clf2.predict(all_data[-1:]), all_target[-1:]

(array([10]), [10])